# Обучение и тест модели MLPModel
1. Подготовка данных ф-ия MLPDataset из dataset.py
2. Определение функции потерь и оптимизатора (MSE - среднеквадратичная ошибка,SGD - стохастический градиентный спуск)
3. Обучение модели
4. Обучение и тестирование на Кросс-Валидации, тк у нас очень маленькая выборка


In [21]:
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, SubsetRandomSampler, TensorDataset

from sklearn.model_selection import KFold

from linRegres import MLPModel
from dataset import MLPDataset,MLPTesDataset


In [23]:
#Импорт модели
model = MLPModel()
#MSE
criterion = nn.MSELoss()  
# Стохастический градиентный спуск
optimizer = optim.SGD(model.parameters(), lr=0.1)  #Епсилон = 0.1 с учетом нашего датасета [-1;1] это приемлемое значение

# Получение датасета и DataLoader
equalDataset = MLPDataset()
dataset = equalDataset['dataset']
train_loader = equalDataset['train_loader']  # Этот DataLoader мы не используем напрямую





In [24]:
# Параметры
k_folds = 5
epochs = 200
# Функция для обучения модели
def train_model(model_q, criterion_q, optimizer_q, train_loader_q, epochs_q):
    model_q.train()  # Перевод модели в режим обучения
    for epoch in range(epochs_q):
        for inputs, labels in train_loader_q:
            optimizer_q.zero_grad()
            outputs = model_q(inputs)
            loss = criterion_q(outputs, labels)
            loss.backward()
            optimizer_q.step()
            
# Функция проверки модели
def test_model(model, criterion, test_loader):
    model.eval()  # Переключаем модель в режим оценки (валидации)
    total_loss = 0.0
    total_samples = 0

    with torch.no_grad():  # Отключаем вычисление градиентов
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item() * inputs.size(0)  # Умножаем потерю на размер пакета для учета разного размера пакетов
            total_samples += inputs.size(0)

    avg_loss = total_loss / total_samples  # Считаем среднюю потерю
    print(f'Средняя потеря на тестовом наборе: {avg_loss}')

In [25]:
# Настройка KFold
kfold = KFold(n_splits=k_folds, shuffle=True)

# Запуск k-фолд кросс-валидации
for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):
    print(f'FOLD {fold+1}')
    print('--------------------------------')
    
    # Сэмплеры для обучающего и тестового набора
    train_subsampler = SubsetRandomSampler(train_ids)
    test_subsampler = SubsetRandomSampler(test_ids)
    
    # DataLoader'ы для текущего разбиения
    train_loader = DataLoader(dataset, batch_size=10, sampler=train_subsampler)
    test_loader = DataLoader(dataset, batch_size=10, sampler=test_subsampler)
    
    # Обучение модели
    train_model(model, criterion, optimizer, train_loader, epochs)
    
    # Тестирование модели
    model.eval()  # Перевод модели в режим оценки
    total_loss = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
    print(f'Test Loss for fold {fold}: {total_loss / len(test_loader)}\n')

FOLD 1
--------------------------------
Test Loss for fold 0: 0.006255189422518015

FOLD 2
--------------------------------
Test Loss for fold 1: 0.003521340200677514

FOLD 3
--------------------------------
Test Loss for fold 2: 0.006525344680994749

FOLD 4
--------------------------------
Test Loss for fold 3: 0.003364887088537216

FOLD 5
--------------------------------
Test Loss for fold 4: 0.0038740055169910192



In [36]:
# Создание тестового набора данных
test_data = MLPTesDataset()
test_loader = test_data['test_loader']

# Проверка модели
test_model(model, criterion, test_loader)

Средняя потеря на тестовом наборе: 12.625688909948803
